<a href="https://colab.research.google.com/github/Shamiul-693/extracts-images-and-text-data-from-a-PDF-and-convert-json-file/blob/main/extracts_images_and_text_data_from_a_user_uploaded_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required libraries
!pip install PyPDF2
!pip install pymupdf

import os
import json
import fitz  # PyMuPDF
from PyPDF2 import PdfReader
from google.colab import files

# Step 1: Upload the PDF file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Step 2: Extract text and images from the PDF
def extract_pdf_data(pdf_path):
    data = {
        "text": [],
        "images": []
    }

    # Extract text using PyPDF2
    reader = PdfReader(pdf_path)
    for page in reader.pages:
        data["text"].append(page.extract_text())

    # Extract images using PyMuPDF
    doc = fitz.open(pdf_path)
    for i in range(len(doc)):
        page = doc.load_page(i)
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]

            # Save the image as a temporary file
            image_filename = f"page_{i+1}_img_{img_index+1}.{image_ext}"
            with open(image_filename, "wb") as img_file:
                img_file.write(image_bytes)

            data["images"].append({
                "page": i + 1,
                "image_file": image_filename
            })

    return data

# Step 3: Process the uploaded PDF file
pdf_data = extract_pdf_data(file_name)

# Step 4: Save extracted data to a JSON file
output_json_file = "extracted_data.json"
with open(output_json_file, "w") as json_file:
    json.dump(pdf_data, json_file, indent=4)

print(f"Data has been extracted and saved to {output_json_file}.")

# Step 5: Download the JSON file
files.download(output_json_file)

# Step 6: Download extracted images (optional)
for image in pdf_data["images"]:
    files.download(image["image_file"])


Saving pd.pdf to pd.pdf
Data has been extracted and saved to extracted_data.json.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# Install the required library
!pip install PyPDF2

from google.colab import files
from PyPDF2 import PdfReader
import ipywidgets as widgets
from IPython.display import display, clear_output

# Step 1: Upload the PDF file
upload_button = widgets.FileUpload(accept='.pdf', multiple=False)
output_box = widgets.Output()

# Function to handle file upload and extract text
# Function to handle file upload and extract text
def extract_text(change):
    clear_output(wait=True)
    output_box.clear_output()

    if upload_button.value:
        # Get the first (and only) uploaded file's information
        file_info = next(iter(upload_button.value.items()))
        file_name = file_info[0]  # The key is now the filename
        file_content = file_info[1]['content']
        with open(file_name, 'wb') as pdf_file:
            pdf_file.write(file_content)

        # Extract text using PyPDF2
        reader = PdfReader(file_name)
        extracted_text = "\n\n".join([page.extract_text() for page in reader.pages])

        # Display extracted text in a text area
        with output_box:
            text_area = widgets.Textarea(
                value=extracted_text,
                placeholder='Extracted text will appear here',
                description='Text:',
                layout=widgets.Layout(width='100%', height='400px')
            )
            display(text_area)

# Observe file upload event
upload_button.observe(extract_text, names='value')

# Display the file upload button and output box
display(widgets.VBox([widgets.Label('Upload a PDF file:'), upload_button, output_box]))
